In [1]:
import urllib.request
import json
from googlesearch import search
import wikipedia
from selenium.webdriver.common.keys import Keys
import time

In [2]:
import bs4 as bs4
import re
from nltk.corpus import stopwords
from selenium import webdriver

In [3]:
import pandas as pd
from random import randint
import numpy as np
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [4]:
with open('data_analysis_indeed.txt') as json_file:
    data = json.load(json_file)

In [6]:
complete_df = pd.read_json(data['complete'])

In [54]:
lnkdf = pd.read_json(data['link_profiles'])

In [76]:
with open('data.txt', mode='w') as outfile:
    json.dump({'rr':rr, 'urls':urls, 'gog_search':gog, 'link_profiles' : scrub_results.to_json(orient='records'), 'complete':complete_df.to_json(orient='records')}, outfile)

In [98]:
with open('data_analysis_indeed.txt', mode='w') as outfile:
    json.dump({'rr':rr, 'gog_result':gog_dataan, 'df':iii.to_json(orient='records'), 'rrdf':inddf.to_json(orient='records'), 'df2': iij.to_json(orient='records')}, outfile)

In [5]:
def time_print(start_time):
    st = start_time
    seconds_tot = round(time.time() - start_time, 0)
    mins = np.floor_divide(seconds_tot, 60)
    seconds = seconds_tot - mins*60
    return (mins, seconds)

In [6]:
def indeed_scrape(search):

    counts = {}
    co = {}
    job_title = []
    idad = []
    ad_count = 0
    url_list =[]
    start_time=time.time()
    
    
    for pageno in range(0,40):
        sear = "https://www.indeed.com/jobs?q=" + search + "&l=07458&explvl=entry_level&radius=50&start=" + str(10*pageno)
        url = urllib.request.urlopen(sear)
        soup = bs4.BeautifulSoup(url)
        this_page_ad_counter = 0

        for adlink in soup.select('a[class = "jobtitle turnstileLink"]'): #select every a element whose class is jobtitle...
                href = adlink['href']
                subURL = "https://www.indeed.com" + href
                subSOUP = bs4.BeautifulSoup(urllib.request.urlopen(subURL))
                url_list.append(subURL)
                
                print("Scraping (" + str(ad_count + 1) + "|id: " + adlink['id'] + ") " + "page: "+ str(pageno) + href[:15] + "...")
                ad_count +=1
                
                if href not in idad:
                    for desc in subSOUP.select('div[class*="jobsearch-JobComponent-description"]'):
                        words = re.split("[ ,.:/?()\n\t\r]", desc.get_text().lower())
                        for word in words:
                            word = word.strip()
                            if word in stopwords.words('english'):
                                continue
                            if word.endswith("'s"):
                                word = word[:-2]
                            if len(word)<2:
                                continue
                            if word in counts:
                                counts[word] +=1
                            else:
                                counts[word] = 1
                    for comp in subSOUP.select('div[class*="jobsearch-CompanyAvatar-cta"]'):
                        cop = comp.get_text()[21:]
                        if cop in co:
                            co[cop][0] +=1
                            print(cop + " Repeated")
                        else:
                            co[cop] =[1,0,0]
                    time.sleep(1)
                    try:
                        for adl in subSOUP.select('div[class *= "icl-Ratings-starsCountWrapper"]'):
       #                     if cop not in co:
                            co[cop][1] = float(adl['aria-label'][0:-9])
                            co[cop][2] = int(adl.get_text()[0:-8].replace(',',''))
                            #print(str(adl['aria-label']))
                    except:
                        co[cop][1] = 'Ratings Error'
                        co[cop][2] = adl
                    for jb in subSOUP.select('h3[class*="jobsearch-JobInfoHeader-title"]'):
                        job = jb.get_text().lower()
                        job_title.append(job)

                    idad.append(href)

                else:
                    continue
                if ad_count%10 == 0:
                    print("--- %s minutes ---" % ((time.time() - start_time)/60))
    word_freq = [] 

    for key, value in counts.items(): 
        word_freq.append((value,key)) 

    word_freq.sort(reverse=True)

    comp_freq = []
    for key, value in co.items(): 
        comp_freq.append((value,key)) 
#Had to remove sort because of strings now present in company dictionary

    return comp_freq, url_list, word_freq

In [10]:
rr, urls, words = indeed_scrape('data+analyst')

Scraping (1|id: sja0) page: 0/pagead/clk?mo=...
Scraping (2|id: sja1) page: 0/pagead/clk?mo=...
Scraping (3|id: sja2) page: 0/pagead/clk?mo=...
Scraping (4|id: sja3) page: 0/pagead/clk?mo=...
Scraping (5|id: sja4) page: 0/pagead/clk?mo=...
Scraping (6|id: jl_b3cb7465234941de) page: 0/company/E-Peop...
Scraping (7|id: jl_23b6d810e3304367) page: 0/rc/clk?jk=23b6...
Scraping (8|id: jl_75335f209c9371fc) page: 0/company/KGS-Te...
Scraping (9|id: jl_939fa67edb3cefbb) page: 0/rc/clk?jk=939f...
Scraping (10|id: jl_ee056d3b186ba1ec) page: 0/company/KGS-Te...
--- 0.37320516506830853 minutes ---
Scraping (11|id: jl_c104c779ce4e6955) page: 0/rc/clk?jk=c104...
Scraping (12|id: jl_7b438840bb78dec7) page: 0/rc/clk?jk=7b43...
Scraping (13|id: jl_f845462fac075333) page: 0/rc/clk?jk=f845...
Scraping (14|id: jl_a5ac6b52ed4197a2) page: 0/company/Ferrer...
Scraping (15|id: jl_4dadde36b1ea325c) page: 0/rc/clk?jk=4dad...
Scraping (16|id: sja15) page: 0/pagead/clk?mo=...
Scraping (17|id: sja16) page: 0/pagead

Scraping (116|id: sja12) page: 6/pagead/clk?mo=...
Cross Border Solutions Repeated
Scraping (117|id: sja13) page: 6/pagead/clk?mo=...
Atrium Staffing Repeated
Scraping (118|id: sja14) page: 6/pagead/clk?mo=...
Computer Enterprises, Inc. (CEI) Repeated
Scraping (119|id: sja15) page: 6/pagead/clk?mo=...
Scraping (120|id: jl_23b44bb76ada487f) page: 7/rc/clk?jk=23b4...
Scraping (121|id: jl_399dab080925c6f9) page: 7/rc/clk?jk=399d...
Scraping (122|id: jl_3af10290453d711d) page: 7/rc/clk?jk=3af1...
Scraping (123|id: jl_68a8d4409f10c35f) page: 7/rc/clk?jk=68a8...
Scraping (124|id: jl_a630d8604e79813a) page: 7/rc/clk?jk=a630...
Scraping (125|id: jl_952db69cb63afd9f) page: 7/rc/clk?jk=952d...
Scraping (126|id: jl_f9671011df04e7c7) page: 7/rc/clk?jk=f967...
Scraping (127|id: jl_172cd941c76bc20f) page: 7/rc/clk?jk=172c...
Scraping (128|id: jl_34e47ecf6aaf90a1) page: 7/rc/clk?jk=34e4...
Scraping (129|id: jl_f11da7d32f3cb281) page: 7/company/PPT/jo...
Scraping (130|id: sja10) page: 7/pagead/clk?mo=

Scraping (225|id: jl_55c10d1034fe0b5b) page: 13/rc/clk?jk=55c1...
Scraping (226|id: jl_4872e5ce95afa12f) page: 13/company/DataAx...
Scraping (227|id: jl_2646d9747e37d496) page: 13/rc/clk?jk=2646...
Scraping (228|id: sja10) page: 13/pagead/clk?mo=...
Atrium Staffing Repeated
Scraping (229|id: sja11) page: 13/pagead/clk?mo=...
Noom Inc. Repeated
Scraping (230|id: sja12) page: 13/pagead/clk?mo=...
DialAmerica Repeated
--- 8.472888815402985 minutes ---
Scraping (231|id: sja13) page: 13/pagead/clk?mo=...
Cross Border Solutions Repeated
Scraping (232|id: sja14) page: 13/pagead/clk?mo=...
Proud Moments ABA Repeated
Scraping (233|id: sja15) page: 13/pagead/clk?mo=...
GM Financial Repeated
Scraping (234|id: jl_8cf5ac1cc4c38f10) page: 14/rc/clk?jk=8cf5...
Scraping (235|id: jl_bf4bf05d45cce9f0) page: 14/rc/clk?jk=bf4b...
Scraping (236|id: jl_a4c2f46c1425e1dc) page: 14/rc/clk?jk=a4c2...
Scraping (237|id: jl_41423755af85584f) page: 14/company/Green-...
Scraping (238|id: jl_a825bbe95cfd398f) page: 1

--- 12.143640116850536 minutes ---
Scraping (331|id: jl_0ba2d11432dc636e) page: 20/rc/clk?jk=0ba2...
Scraping (332|id: jl_dfaa78b64925f27e) page: 20/rc/clk?jk=dfaa...
JP Morgan Chase Repeated
Scraping (333|id: jl_10ec234b893e4a57) page: 20/rc/clk?jk=10ec...
Scraping (334|id: jl_17081ff1c7f8d4ae) page: 20/rc/clk?jk=1708...
Scraping (335|id: jl_d87f9b9e6225e755) page: 20/rc/clk?jk=d87f...
Cognizant Technology Solutions Repeated
Scraping (336|id: jl_6c34ec87324f409c) page: 20/rc/clk?jk=6c34...
Scraping (337|id: jl_78a0f58133f4dcc1) page: 20/rc/clk?jk=78a0...
Scraping (338|id: jl_5e07a2d52cc410a6) page: 20/rc/clk?jk=5e07...
Scraping (339|id: sja10) page: 20/pagead/clk?mo=...
Atrium Staffing Repeated
Scraping (340|id: sja11) page: 20/pagead/clk?mo=...
Greater Bergen Community Action, Inc. Repeated
--- 12.475360488891601 minutes ---
Scraping (341|id: sja12) page: 20/pagead/clk?mo=...
Noom Inc. Repeated
Scraping (342|id: sja13) page: 20/pagead/clk?mo=...
GM Financial Repeated
Scraping (343|id

Scraping (438|id: jl_79cdfeeabfaea195) page: 26/rc/clk?jk=79cd...
Scraping (439|id: jl_55490489606e765a) page: 26/company/Sterli...
Scraping (440|id: jl_894c7353f7f13eb8) page: 26/rc/clk?jk=894c...
Scraping (441|id: jl_557274399c2aefbb) page: 26/rc/clk?jk=5572...
Scraping (442|id: jl_80e433a20dc6b184) page: 26/rc/clk?jk=80e4...
Scraping (443|id: sja13) page: 26/pagead/clk?mo=...
Workforce Opportunity Services Repeated
Scraping (444|id: sja14) page: 26/pagead/clk?mo=...
Healthfirst Repeated
Scraping (445|id: sja15) page: 26/pagead/clk?mo=...
Cross Border Solutions Repeated
Scraping (446|id: jl_34dcf21b63ff4759) page: 27/rc/clk?jk=34dc...
Scraping (447|id: jl_62107e7ed5f9bef3) page: 27/rc/clk?jk=6210...
Macy's Repeated
Scraping (448|id: jl_ad09186f43c5564b) page: 27/rc/clk?jk=ad09...
Scraping (449|id: jl_4d232faa22bcac5c) page: 27/rc/clk?jk=4d23...
Scraping (450|id: jl_23414ec13c9f7788) page: 27/rc/clk?jk=2341...
--- 16.53433924118678 minutes ---
Scraping (451|id: jl_9d1260d0a1d9269a) pa

Scraping (546|id: jl_340544b2a8a10726) page: 33/rc/clk?jk=3405...
New York City DEPT OF HEALTH/MENTAL HYGIENE Repeated
Scraping (547|id: jl_4aaf425092728c98) page: 33/rc/clk?jk=4aaf...
Scraping (548|id: jl_64a9697bd67784e2) page: 33/rc/clk?jk=64a9...
Scraping (549|id: jl_8314a096bf4f300f) page: 33/rc/clk?jk=8314...
Scraping (550|id: jl_98af3eeda461d449) page: 33/company/Statis...
--- 20.35442292292913 minutes ---
Scraping (551|id: jl_49998d4bf73cec2b) page: 33/rc/clk?jk=4999...
Scraping (552|id: jl_9d90a8a243a23426) page: 33/rc/clk?jk=9d90...
Scraping (553|id: jl_73ec9fb205ff520e) page: 33/rc/clk?jk=73ec...
Scraping (554|id: jl_a54254d7f2709b82) page: 33/rc/clk?jk=a542...
Scraping (555|id: sja10) page: 33/pagead/clk?mo=...
Workforce Opportunity Services Repeated
Scraping (556|id: sja11) page: 33/pagead/clk?mo=...
PartsWatch Solutions LLC Repeated
Scraping (557|id: sja12) page: 33/pagead/clk?mo=...
Cross Border Solutions Repeated
Scraping (558|id: sja13) page: 33/pagead/clk?mo=...
Atriu

Scraping (651|id: sja13) page: 39/pagead/clk?mo=...
Noom Inc. Repeated
Scraping (652|id: sja14) page: 39/pagead/clk?mo=...
Greater Bergen Community Action, Inc. Repeated
Scraping (653|id: sja15) page: 39/pagead/clk?mo=...
Cross Border Solutions Repeated
Scraping (654|id: sja16) page: 39/pagead/clk?mo=...
DialAmerica Repeated


In [13]:
len(rr)

294

In [24]:
len(gog)

127

In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
def google_search(comp_freq):
    pages = {}
    start_time = time.time()

    for ctprint, com_s in enumerate([i[1] for i in comp_freq]):
        try:
            for wp in search(com_s + " linkedin", stop=1):
                if '.com/company/' in wp:
                    pages[wp] = com_s
                if ctprint%5 == 0:
                    print('Webpages Added: ' + str(ctprint))
                    print('---%s minutes---' % round((time.time() - start_time)/60, 1))
        except:
            print(com_s + ' Page Not Available')
    
    return pages

In [9]:
def link_scrape(pages):

    start_time = time.time()
    driver = webdriver.Firefox()
    driver.get('https://www.linkedin.com/')
    user = driver.find_element_by_name('session_key')
    pasw = driver.find_element_by_name('session_password')

    user.send_keys('istobbe@u.rochester.edu')
    time.sleep(0.7)
    pasw.send_keys('99000996')
    pasw.send_keys(Keys.RETURN)
    time.sleep(1)

    ind_path = ['/html/body/div[5]/div[6]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[2]',
               '/html/body/div[5]/div[6]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[3]']
    size_path = ['/html/body/div[5]/div[6]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[3]',
                '/html/body/div[5]/div[6]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dd[4]']
    phone_path = '/html/body/div[5]/div[6]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dt[2]'
    
    industries = []

    for ctimer, url in enumerate(pages):
        if '/about/' not in url:
            url2 = url + '/about/'
            driver.get(url2)
            try:
                element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, phone_path)))
                if driver.find_element_by_xpath(phone_path).text == 'Phone':
                    try:
                        ind = driver.find_element_by_xpath(ind_path[1]).text
                        size = driver.find_element_by_xpath(size_path[1]).text
                        industries.append((pages[url], ind, size))
                    except NoSuchElementException:
                        time.sleep(5)
                        ind = driver.find_element_by_xpath(ind_path[1]).text
                        size = driver.find_element_by_xpath(size_path[1]).text
                        industries.append((pages[url], ind, size))
                else:
                    try:
                        time.sleep(1.5)
                        ind = driver.find_element_by_xpath(ind_path[0]).text
                        size = driver.find_element_by_xpath(size_path[0]).text
                        industries.append((pages[url], ind, size))
                    except NoSuchElementException:
                        time.sleep(6.5)
                        try:
                            ind = driver.find_element_by_xpath(ind_path[0]).text
                            size = driver.find_element_by_xpath(size_path[0]).text
                            industries.append((pages[url], ind, size))
                        except NoSuchElementException:
                            print('Element Find Issue')
                            industries.append((pages[url], 'Element Find Issue', 'Element Find Issue'))
            except TimeoutException:
                continue
        if ctimer%5 == 0:
            print("--- %d minutes and %d seconds --- for " % time_print(start_time) + str(ctimer) + " sites")
    driver.quit()
    industries = pd.DataFrame(industries, columns=['company', 'industry', 'size'])
    return industries

In [11]:
import itertools
chkr = dict(itertools.islice(data['gog_result'].items(), 0, 30))
dfff = link_scrape(chkr)

--- 0 minutes and 17 seconds --- for 0 sites
--- 0 minutes and 34 seconds --- for 5 sites
--- 0 minutes and 53 seconds --- for 10 sites
--- 1 minutes and 14 seconds --- for 15 sites
--- 1 minutes and 36 seconds --- for 20 sites
--- 2 minutes and 17 seconds --- for 25 sites


--- 0.3 minutes --- for 0 sites
--- 0.8 minutes --- for 5 sites


In [33]:
gog_dataan = google_search(rr)

Webpages Added: 0
---0.0 minutes---
Webpages Added: 5
---0.3 minutes---
Webpages Added: 10
---0.6 minutes---
RWJBarnabas Health Page Not Available
Webpages Added: 15
---0.8 minutes---
Webpages Added: 20
---1.1 minutes---
Webpages Added: 25
---1.3 minutes---
Webpages Added: 30
---1.6 minutes---
Webpages Added: 35
---1.8 minutes---
Webpages Added: 40
---2.1 minutes---
Webpages Added: 45
---2.3 minutes---
Webpages Added: 50
---2.6 minutes---
Webpages Added: 55
---2.8 minutes---
Webpages Added: 60
---3.1 minutes---
Webpages Added: 65
---3.3 minutes---
Webpages Added: 70
---3.6 minutes---
Webpages Added: 75
---3.8 minutes---
Webpages Added: 80
---4.1 minutes---
Webpages Added: 85
---4.3 minutes---
Webpages Added: 90
---4.6 minutes---
Webpages Added: 95
---4.8 minutes---
Webpages Added: 100
---5.1 minutes---
Webpages Added: 105
---5.3 minutes---
Webpages Added: 110
---5.5 minutes---
Webpages Added: 115
---5.8 minutes---
Webpages Added: 120
---6.0 minutes---
Webpages Added: 125
---6.3 minutes

In [62]:
rr

[([27, 3.2, 1951], 'DialAmerica'),
 ([21, 3.6, 5], 'Proud Moments ABA'),
 ([19, 3.6, 17], 'Computer Enterprises, Inc. (CEI)'),
 ([24, 3.9, 104], 'Atrium Staffing'),
 ([19, 3.5, 30], 'Workforce Opportunity Services'),
 ([1, 0, 0], 'E Peopletech'),
 ([1, 4.0, 5727], 'KPMG'),
 ([2, 0, 0], 'KGS Technology Group, Inc'),
 ([1, 2.2, 45], 'Asembia LLC'),
 ([2, 4.8, 33], 'KGS Technology Group Inc'),
 ([2, 3.9, 195], 'Dow Jones'),
 ([1, 0, 0], 'Colas Inc. (100)'),
 ([1, 3.9, 1395], 'RWJBarnabas Health'),
 ([1, 4.1, 515], 'Ferrero USA'),
 ([1, 0, 0], 'Protomos'),
 ([26, 3.7, 596], 'GM Financial'),
 ([40, 2.8, 5], 'Cross Border Solutions'),
 ([18, 3.3, 11], 'Greater Bergen Community Action, Inc.'),
 ([21, 4.3, 9], 'Noom Inc.'),
 ([3, 0, 0], 'NRZ Entertainment LLC.'),
 ([12, 3.9, 513], 'Barnes & Noble College'),
 ([1, 3.7, 4379], 'Quest Diagnostics'),
 ([1, 0, 0], 'Primesoft Consulting Services Inc.'),
 ([1, 3.2, 51], 'TRANZACT'),
 ([1, 0, 0], 'Direct Client (Only W2 or Per Diem)'),
 ([2, 4.0, 5], 

In [97]:
iij = link_scrape(dict(itertools.islice(gog_dataan.items(), 100, 267)))

--- 0 minutes and 11 seconds --- for 0 sites
--- 0 minutes and 31 seconds --- for 5 sites
--- 0 minutes and 50 seconds --- for 10 sites
--- 1 minutes and 11 seconds --- for 15 sites
--- 1 minutes and 32 seconds --- for 20 sites
--- 2 minutes and 30 seconds --- for 30 sites
--- 2 minutes and 49 seconds --- for 35 sites
--- 3 minutes and 9 seconds --- for 40 sites
--- 3 minutes and 28 seconds --- for 45 sites
--- 3 minutes and 46 seconds --- for 50 sites
--- 4 minutes and 25 seconds --- for 55 sites
--- 4 minutes and 43 seconds --- for 60 sites
--- 5 minutes and 1 seconds --- for 65 sites
--- 5 minutes and 21 seconds --- for 70 sites
--- 5 minutes and 40 seconds --- for 75 sites
--- 6 minutes and 0 seconds --- for 80 sites
Element Find Issue
--- 6 minutes and 26 seconds --- for 85 sites
Element Find Issue
--- 6 minutes and 55 seconds --- for 90 sites
--- 7 minutes and 16 seconds --- for 95 sites
--- 7 minutes and 37 seconds --- for 100 sites
--- 8 minutes and 0 seconds --- for 105 sites


In [56]:
iii

,company,industry,size
0,DialAmerica,Outsourcing/Offshoring,"1,001-5,000 employees"
1,Proud Moments ABA,Individual & Family Services,"501-1,000 employees"
2,"Computer Enterprises, Inc. (CEI)",Information Technology and Services,"501-1,000 employees"
3,Atrium Staffing,Staffing and Recruiting,201-500 employees
4,Workforce Opportunity Services,Staffing and Recruiting,201-500 employees
5,E Peopletech,Information Technology and Services,"501-1,000 employees"
6,KPMG,Accounting,"10,001+ employees"
7,KGS Technology Group Inc,Computer Software,51-200 employees
8,Asembia LLC,Pharmaceuticals,"501-1,000 employees"
9,Dow Jones,Online Media,"1,001-5,000 employees"


In [31]:
driver = webdriver.Firefox()
driver.get('https://www.linkedin.com/')
user = driver.find_element_by_name('session_key')
pasw = driver.find_element_by_name('session_password')

user.send_keys('istobbe@u.rochester.edu')
time.sleep(0.7)
pasw.send_keys('99000996')
pasw.send_keys(Keys.RETURN)
time.sleep(1)

phone_path = '/html/body/div[5]/div[6]/div[3]/div/div[2]/div/div[2]/div[1]/section/dl/dt[2]'
industries = []

driver.get('https://www.linkedin.com/company/gm-financial/about/')
time.sleep(randint(1, 4))

try:
    phone = driver.find_element_by_xpath(phone_path).text
except NoSuchElementException:
    print(Error)
print(phone)
driver.quit()


Industry


In [107]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [17]:
sum([i[0] for i in companies])

397

In [63]:
inddf = pd.DataFrame(rr)

In [64]:
inddf['job_postings'] = inddf[0].apply(lambda x: x[0])
inddf['indeed_rating'] = inddf[0].apply(lambda x: x[1])
inddf['indeed_reviews'] = inddf[0].apply(lambda x: x[2])

In [65]:
inddf.drop(columns = 0, inplace=True)

In [117]:
indlist = {}
for i, ind in enumerate(complete_df['industry']):
    if ind in indlist:
        indlist[ind] += complete_df['job_postings'][i]
    else:
        indlist[ind] = complete_df['job_postings'][i]

In [71]:
inddf.rename(columns={1:'company'}, inplace=True)

In [94]:
pd.merge(iii, inddf, on='company')

,company,industry,size,job_postings,indeed_rating,indeed_reviews
0,DialAmerica,Outsourcing/Offshoring,"1,001-5,000 employees",27,3.2,1951
1,Proud Moments ABA,Individual & Family Services,"501-1,000 employees",21,3.6,5
2,"Computer Enterprises, Inc. (CEI)",Information Technology and Services,"501-1,000 employees",19,3.6,17
3,Atrium Staffing,Staffing and Recruiting,201-500 employees,24,3.9,104
4,Workforce Opportunity Services,Staffing and Recruiting,201-500 employees,19,3.5,30
5,E Peopletech,Information Technology and Services,"501-1,000 employees",1,0.0,0
6,KPMG,Accounting,"10,001+ employees",1,4.0,5727
7,KGS Technology Group Inc,Computer Software,51-200 employees,2,4.8,33
8,Asembia LLC,Pharmaceuticals,"501-1,000 employees",1,2.2,45
9,Dow Jones,Online Media,"1,001-5,000 employees",2,3.9,195


In [77]:
gp_df.get_group('Financial Services')

,company,industry,size,job_postings,indeed_rating,indeed_reviews
13,GM Financial,Financial Services,"5,001-10,000 employees",26,3.7,596
14,Cross Border Solutions,Financial Services,51-200 employees,40,2.8,5
51,Citi,Financial Services,"10,001+ employees",2,3.9,16096
53,MassMutual,Financial Services,"10,001+ employees",1,3.7,1000
90,Zelis Payments,Financial Services,51-200 employees,1,1.9,21


In [78]:
gp_df.get_group('Internet')

,company,industry,size,job_postings,indeed_rating,indeed_reviews
23,Facebook,Internet,"10,001+ employees",2,4.2,469
42,CareDox,Internet,11-50 employees,1,0.0,0
48,"Audible, Inc.",Internet,"1,001-5,000 employees",1,3.6,36010
81,Criteo,Internet,"1,001-5,000 employees",2,4.0,45
93,Digitalogy Consulting,Internet,11-50 employees,1,0.0,0


In [79]:
gp_df.get_group('Hospital & Health Care')

,company,industry,size,job_postings,indeed_rating,indeed_reviews
18,Quest Diagnostics,Hospital & Health Care,"10,001+ employees",1,3.7,4379
22,HACKENSACK UNIVERSITY MEDICAL CENTER,Hospital & Health Care,"10,001+ employees",1,4.1,423
31,Montefiore Medical Center,Hospital & Health Care,"10,001+ employees",3,4.1,1233
50,CareCube,Hospital & Health Care,2-10 employees,1,0.0,0
60,Healthfirst,Hospital & Health Care,"1,001-5,000 employees",9,3.5,492
63,Zufall Health,Hospital & Health Care,51-200 employees,1,0.0,0
65,North American Partners in Anesthesia,Hospital & Health Care,"1,001-5,000 employees",1,3.1,25
96,Summit Health Management,Hospital & Health Care,51-200 employees,1,2.9,24
97,NewYork-Presbyterian Hospital,Hospital & Health Care,"10,001+ employees",1,4.2,1608


In [80]:
gp_df.get_group('Outsourcing/Offshoring')

,company,industry,size,job_postings,indeed_rating,indeed_reviews
0,DialAmerica,Outsourcing/Offshoring,"1,001-5,000 employees",27,3.2,1951


In [81]:
gp_df.describe()

job_postings                              \
                                           count       mean        std   min   
industry                                                                       
Accounting                                   1.0   1.000000        NaN   1.0   
Apparel & Fashion                            1.0   1.000000        NaN   1.0   
Automotive                                   1.0   1.000000        NaN   1.0   
Banking                                      1.0   1.000000        NaN   1.0   
Chemicals                                    1.0   1.000000        NaN   1.0   
Civic & Social Organization                  1.0   1.000000        NaN   1.0   
Computer Software                            6.0   1.333333   0.516398   1.0   
Construction                                 2.0   1.500000   0.707107   1.0   
Consumer Goods                               3.0   1.666667   0.577350   1.0   
Entertainment                                2.0   1.000000   0.000000   1.0   
Financial Services                           5.0  14.000000  18.041619   1.0   
Fine Art                                     1.0   1.000000        NaN   1.0   
Food & Beverages                             1.0   1.000000        NaN   1.0   
Government Administration                    5.0   1.400000   0.894427   1.0   
Health, Wellness and Fitness                 2.0  11.000000  14.142136   1.0   
Hospital & Health Care                       9.0   2.111111   2.666667   1.0   
Individual & Family Services                 1.0  21.000000        NaN  21.0   
Information Services                         2.0   1.000000   0.000000   1.0   
Information Technology and Services         11.0   3.909091   6.331738   1.0   
Insurance                                    2.0   1.000000   0.000000   1.0   
Internet                                     5.0   1.400000   0.547723   1.0   
Luxury Goods & Jewelry                       1.0   1.000000        NaN   1.0   
Management Consulting                        1.0   1.000000        NaN   1.0   
Marketing and Advertising                    8.0   1.125000   0.353553   1.0   
Medical Devices                              3.0   1.000000   0.000000   1.0   
Newspapers                                   1.0   1.000000        NaN   1.0   
Nonprofit Organization Management            2.0   9.500000  12.020815   1.0   
Online Media                                 1.0   2.000000        NaN   2.0   
Outsourcing/Offshoring                       1.0  27.000000        NaN  27.0   
Pharmaceuticals                              2.0   1.000000   0.000000   1.0   
Professional Training & Coaching             1.0   1.000000        NaN   1.0   
Real Estate                                  1.0   1.000000        NaN   1.0   
Retail                                       3.0   4.666667   6.350853   1.0   
Security and Investigations                  1.0   1.000000        NaN   1.0   
Semiconductors                               1.0   1.000000        NaN   1.0   
Staffing and Recruiting                      8.0   6.500000   9.365591   1.0   
Telecommunications                           1.0   2.000000        NaN   2.0   
Wine and Spirits                             1.0   1.000000        NaN   1.0   

                                                              indeed_rating  \
                                       25%   50%    75%   max         count   
industry                                                                      
Accounting                            1.00   1.0   1.00   1.0           1.0   
Apparel & Fashion                     1.00   1.0   1.00   1.0           1.0   
Automotive                            1.00   1.0   1.00   1.0           1.0   
Banking                               1.00   1.0   1.00   1.0           1.0   
Chemicals                             1.00   1.0   1.00   1.0           1.0   
Civic & Social Organization           1.00   1.0   1.00   1.0           1.0   
Computer Software                     1.00  

In [86]:
gp_df['job_postings'].describe()

,count,mean,std,min,25%,50%,75%,max
industry,,,,,,,,
Accounting,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
Apparel & Fashion,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
Automotive,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
Banking,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
Chemicals,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
Civic & Social Organization,1.0,1.000000,NaN,1.0,1.00,1.0,1.00,1.0
Computer Software,6.0,1.333333,0.516398,1.0,1.00,1.0,1.75,2.0
Construction,2.0,1.500000,0.707107,1.0,1.25,1.5,1.75,2.0
Consumer Goods,3.0,1.666667,0.577350,1.0,1.50,2.0,2.00,2.0


In [5]:
urlls = data['urls']

In [7]:
urlls[10]

'https://www.indeed.com/company/CloudServiceTek/jobs/Entry-Level-Business-Analyst-bce986d932473485?fccid=5de1fafcbe6b6997&vjs=3'

In [8]:
df1 = pd.read_json(data['df'])

KeyError: 'df'

In [7]:
df2 = pd.read_json(data['df2'])

In [8]:
indeed_df = pd.read_json(data['rrdf'])

In [9]:
com_df = pd.concat([df1,df2])

In [10]:
woah_df = pd.merge(com_df, indeed_df, on='company')

In [11]:
indlist = {}
for i, ind in enumerate(woah_df['industry']):
    if ind in indlist:
        indlist[ind] += woah_df['job_postings'][i]
    else:
        indlist[ind] = woah_df['job_postings'][i]

In [12]:
ind_jobs = pd.DataFrame.from_dict(indlist, orient='index', columns=['Job_Postings'])

In [13]:
ind_jobs.sort_values(by='Job_Postings', ascending=False)

,Job_Postings
Financial Services,90
Information Technology and Services,70
Staffing and Recruiting,56
Hospital & Health Care,37
Outsourcing/Offshoring,27
"Health, Wellness and Fitness",23
Retail,23
Nonprofit Organization Management,22
Individual & Family Services,21
Marketing and Advertising,18
